# <center>Masakhane - Machine Translation for African Languages (Using JoeyNMT)</center>
## <leftalign> Author : Ari Ramkilowan</leftalign>
## <leftalign> Language Pair : English - isiZulu</leftalign>
## <leftalign> Corpus : JW300 </leftalign>

<hr>

## Install JoeyNMT

In [0]:
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 2184, done.
remote: Total 2184 (delta 0), reused 0 (delta 0), pack-reused 2184
Receiving objects: 100% (2184/2184), 2.58 MiB | 4.48 MiB/s, done.
Resolving deltas: 100% (1521/1521), done.
Processing /content/joeynmt
     |████████████████████████████████| 266kB 16.2MB/s 
     |████████████████████████████████| 307kB 40.7MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 215kB 41.1MB/s 
     |████████████████████████████████| 737kB 44.2MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=71902 sha256=00ffe5c04aa3d9faab1d40630911047ff94093f60c14a19fd8c298fc730a62e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-pgpcy60v/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44104 sha256=1af8880955a8fa687c75ed9


## Mount Google Drive


In [0]:
# If running on Google Colab - mount google drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## What GPU do we have available today ?

In [0]:
# keep trying until we get a beefier GPU : T4 > V100> P100 > K80

import torch
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

device_num = torch.cuda.current_device()
torch.cuda.get_device_name(device_num)
# torch.cuda.is_available()

'Tesla P100-PCIE-16GB'

## Set your source, target language and experiment tag


In [0]:
import os
import numpy as np
import pandas as pd

source_language = "en"
target_language = "zu" 
lc = True  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted
vocab_size=4000
corpus = "JW300"

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag
os.environ["vocab_size"] = str(vocab_size)
os.environ["corpus"] = corpus

In [0]:
# This will save it to a folder in our gdrive instead!
# !mkdir -p "/content/drive/My Drive/masakhane/$src-$trg-$tag"
gdrive_path = f"/content/drive/My Drive/masakhane/{source_language}-{target_language}-{tag}/"
os.environ["gdrive_path"] = gdrive_path
! echo $gdrive_path

/content/drive/My Drive/masakhane/en-zu-baseline/


In [0]:
# create path to joeynmt executables scripts, configs etc

joey_path = f"/content/joeynmt"
os.environ["joey_path"] = joey_path
! ls $joey_path/configs

iwslt14_deen_bpe.yaml		   transformer_reverse.yaml
iwslt_deen_bahdanau.yaml	   transformer_small.yaml
iwslt_envi_luong.yaml		   transformer_wmt17_ende.yaml
iwslt_envi_xnmt.yaml		   transformer_wmt17_lven.yaml
reverse.yaml			   wmt_ende_best.yaml
small.yaml			   wmt_ende_default.yaml
transformer_copy.yaml		   wmt_lven_best.yaml
transformer_enzu4000const_lr.yaml  wmt_lven_default.yaml
transformer_iwslt14_deen_bpe.yaml


## Download the global test set.

***(This changes from time to time, do this just to make sure you have the most recent version)***


In [0]:
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en
! mv test.en-$trg.en test.en

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2019-11-12 14:46:36--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.02s   

2019-11-12 14:46:36 (12.8 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2019-11-12 14:46:39--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-zu.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [0]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
blanks = [] # sometimes blank lines creep innto test set - store which lines these are
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    if len(line)<=1:
      blanks.append(j)
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))
print(f'There are {len(blanks)} blank lines in the test set')

Loaded 3571 global test sentences to filter from the training/dev data.
There are 0 blank lines in the test set


In [0]:
# filter test set

source_file = f"test.{source_language}"
target_file = f"test.{target_language}"

source = []
target = []

with open(source_file) as f:
  source = f.readlines()
            
with open(target_file) as f:
  target = f.readlines()

df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])

# remove trailing newline chars
df['source_sentence'] = df['source_sentence'].str.rstrip('" \n')
df['target_sentence'] = df['target_sentence'].str.rstrip('" \n')

# remove leading newline chars
df['source_sentence'] = df['source_sentence'].str.lstrip('"')
df['target_sentence'] = df['target_sentence'].str.lstrip('"')

# remove rows with really short sentences
df = df[~(df['source_sentence'].str.len() <8)] # remove rows wher esource text len <8 characters
df = df[~(df['target_sentence'].str.len() <8)] # remove rows wher esource text len <8 characters

# save the filtered test set
df['source_sentence'].to_csv(f'{source_file}', index=False, header=False, doublequote=False)
df['target_sentence'].to_csv(f'{target_file}', index=False, header=False, doublequote=False)

In [0]:
# copy test sets to gdrive
! cp test.$src "$gdrive_path"
! cp test.$trg "$gdrive_path"
! cp test.$src-any.$src "$gdrive_path"

## Import prepared dataset

In [0]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
# This csv has extra columns added but no preprocessing done. all preprocessing should be captured in the NMT modelling notebook

input_file = f"{gdrive_path}/{source_language}-{target_language}-{corpus}-new.csv"
df = pd.read_csv(input_file)

In [0]:
df.head()

,source_sentence,target_sentence
0,Unity in Worship ​ — What Should It Mean for Y...,Ubunye Ekukhulekeleni — Kumelwe Busho Ukuthini...
1,As we draw closer and closer to the end of thi...,Njengoba silokhu sisondela ekupheleni kwalesim...
2,"So it is more difficult , and yet more urgent ...","Ngakho kunzima ngokwengeziwe , futhi kuphuthum..."
3,The new book United in Worship of the Only Tru...,Incwadi entsha ethi Ubunye Ekukhulekeleni Okuw...
4,"Please send me , postpaid , the 192 - page , h...",Ngicela ningithumele incwadi enoqwembe oluqini...


In [0]:
# How many samples
size = len(df)
print(f"\n {size} samples in original text")
  


 1062534 samples in original text


## Preprocess input data

In [0]:
## Preprocessing - Step 1 : Drop NaNs

df_pp = df.dropna()
df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping all NaNs")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1053497 entries, 0 to 1062533
Data columns (total 2 columns):
source_sentence    1053497 non-null object
target_sentence    1053497 non-null object
dtypes: object(2)
memory usage: 402.1 MB

 9037(0.85 %) samples removed by dropping all NaNs


In [0]:
## Preprocessing - Step 2a : Drop all duplicates in Source (en) text

df_pp = df_pp.drop_duplicates(subset='source_sentence')
df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping Source sentence duplicates")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 970498 entries, 0 to 1062533
Data columns (total 2 columns):
source_sentence    970498 non-null object
target_sentence    970498 non-null object
dtypes: object(2)
memory usage: 430.5 MB

 82999(7.88 %) samples removed by dropping Source sentence duplicates


In [0]:
## Preprocessing - Step 2b : Drop all duplicates in Target (zu) text

df_pp = df_pp.drop_duplicates(subset='target_sentence')
df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping Target sentence duplicates")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 965453 entries, 0 to 1062533
Data columns (total 2 columns):
source_sentence    965453 non-null object
target_sentence    965453 non-null object
dtypes: object(2)
memory usage: 465.7 MB

 5045(0.52 %) samples removed by dropping Target sentence duplicates


In [0]:
##  Preprocessing - Step 3 : Remove all numeric entries

pattern = r"([0-9]*\.?[0-9]*)"  # catch integers and decimals
import re
r = re.compile(pattern)

In [0]:
%%time
##  Preprocessing - Step 3a : Remove all numeric entries - Source text

df_pp['source_sentence'] = df_pp['source_sentence'].str.replace(pattern,"")
df_pp['source_sentence'] = df_pp['source_sentence'].replace("",np.nan)

df_pp = df_pp.dropna()
df_pp.info(memory_usage='deep')
new_size = len(df_pp)

print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping nummeric entries from source text")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 965271 entries, 0 to 1062533
Data columns (total 2 columns):
source_sentence    965271 non-null object
target_sentence    965271 non-null object
dtypes: object(2)
memory usage: 419.3 MB

 182(0.02 %) samples removed by dropping nummeric entries from source text
CPU times: user 16.8 s, sys: 116 ms, total: 16.9 s
Wall time: 16.9 s


In [0]:
%%time
##  Preprocessing - Step 3b : Remove all numeric entries - Target text

df_pp['target_sentence'] = df_pp['target_sentence'].str.replace(r,"")
df_pp['target_sentence'] = df_pp['target_sentence'].replace("",np.nan)

df_pp = df_pp.dropna()
df_pp.info(memory_usage='deep')
new_size = len(df_pp)

print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping nummeric entries from target text")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 965271 entries, 0 to 1062533
Data columns (total 2 columns):
source_sentence    965271 non-null object
target_sentence    965271 non-null object
dtypes: object(2)
memory usage: 380.8 MB

 0(0.00 %) samples removed by dropping nummeric entries from target text
CPU times: user 17.4 s, sys: 106 ms, total: 17.5 s
Wall time: 17.5 s


#### Preprocessing - Step 4 :Get length of sentences and then drop really short sentences


In [0]:
%%time
# add length columns


df_pp['source_ch_len'] = df_pp['source_sentence'].str.len()
df_pp['source_w_len'] = [len(text.split()) for text in df_pp['source_sentence']] 
df_pp['target_ch_len'] = df_pp['target_sentence'].str.len()
df_pp['target_w_len'] = [len(text.split()) for text in df_pp['target_sentence']] 
df_pp.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 965271 entries, 0 to 1062533
Data columns (total 6 columns):
source_sentence    965271 non-null object
target_sentence    965271 non-null object
source_ch_len      965271 non-null int64
source_w_len       965271 non-null int64
target_ch_len      965271 non-null int64
target_w_len       965271 non-null int64
dtypes: int64(4), object(2)
memory usage: 410.3 MB
CPU times: user 4.5 s, sys: 28 ms, total: 4.53 s
Wall time: 4.53 s


In [0]:
# # character len distrn - source text - 
# df_pp['source_ch_len'].value_counts().sort_index()

In [0]:
# # character len distrn - target text
# df_pp['target_ch_len'].value_counts().sort_index()

In [0]:
## how many rows with source text <=2chars and what do they look like ?

In [0]:
# # how many single character sentences from source ?
# f"{df_pp['source_ch_len'].value_counts()[1]} single character source sentences"

# df_pp[df_pp['source_ch_len']<=1]

In [0]:
# # how many 2-character sentences from source ?
# f"{df_pp['source_ch_len'].value_counts()[2]} 2-character source sentences"

# df_pp[df_pp['source_ch_len']==2]

In [0]:
##  Preprocessing - Step 4a :  drop everything where the ch_len <=8 in source text

df_pp = df_pp[~(df_pp['source_ch_len'] <=8) ]

df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping rows with source sentences <= 8 characters")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 958999 entries, 0 to 1062533
Data columns (total 6 columns):
source_sentence    958999 non-null object
target_sentence    958999 non-null object
source_ch_len      958999 non-null int64
source_w_len       958999 non-null int64
target_ch_len      958999 non-null int64
target_w_len       958999 non-null int64
dtypes: int64(4), object(2)
memory usage: 409.2 MB

 6272(0.65 %) samples removed by dropping rows with source sentences <= 8 characters


In [0]:
##  Preprocessing - Step 4b :  drop everything where the ch_len <=8 in target text

df_pp = df_pp[~(df_pp['target_ch_len'] <=8) ]

df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping rows with target sentences <= 8 characters")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 958286 entries, 0 to 1062533
Data columns (total 6 columns):
source_sentence    958286 non-null object
target_sentence    958286 non-null object
source_ch_len      958286 non-null int64
source_w_len       958286 non-null int64
target_ch_len      958286 non-null int64
target_w_len       958286 non-null int64
dtypes: int64(4), object(2)
memory usage: 409.1 MB

 713(0.07 %) samples removed by dropping rows with target sentences <= 8 characters


In [0]:
%%time
##  Preprocessing - Step 5 :  remove the extra "
df_pp['source_sentence'] = df_pp['source_sentence'].map(lambda x: x.lstrip('"').rstrip('"'))
df_pp['target_sentence'] = df_pp['target_sentence'].map(lambda x: x.lstrip('"').rstrip('"'))


df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping rows with extra quotes")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 958286 entries, 0 to 1062533
Data columns (total 6 columns):
source_sentence    958286 non-null object
target_sentence    958286 non-null object
source_ch_len      958286 non-null int64
source_w_len       958286 non-null int64
target_ch_len      958286 non-null int64
target_w_len       958286 non-null int64
dtypes: int64(4), object(2)
memory usage: 409.1 MB

 0(0.00 %) samples removed by dropping rows with extra quotes
CPU times: user 2.39 s, sys: 7.98 ms, total: 2.39 s
Wall time: 2.41 s


In [0]:
%%time
##  Preprocessing - Step 6 :  remove text from test set

with open(f"{gdrive_path}/test.en-any.en") as f:
    rows = f.readlines()
test_set_en = [row.strip() for row in rows]


df_pp = df_pp[~df_pp['source_sentence'].str.strip().isin(test_set_en)]

df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping rows from test set")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 963871 entries, 0 to 1062533
Data columns (total 6 columns):
source_sentence    963871 non-null object
target_sentence    963871 non-null object
source_ch_len      963871 non-null int64
source_w_len       963871 non-null int64
target_ch_len      963871 non-null int64
target_w_len       963871 non-null int64
dtypes: int64(4), object(2)
memory usage: 409.9 MB

 1068(0.11 %) samples removed by dropping rows from test set
CPU times: user 3 s, sys: 57 ms, total: 3.05 s
Wall time: 3.06 s
Compiler : 325 ms


## create dev df 

In [0]:
df_dev = df_pp[['source_sentence', 'target_sentence']]
# Shuffle the data to remove bias in dev set selection.
seed=42
df_dev = df_dev.sample(frac=1, random_state=seed).reset_index(drop=True)
df_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958286 entries, 0 to 958285
Data columns (total 2 columns):
source_sentence    958286 non-null object
target_sentence    958286 non-null object
dtypes: object(2)
memory usage: 14.6+ MB


## Create train test dev sets

In [0]:
%%time
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_dev["source_sentence"] = df_dev["source_sentence"].str.lower()
    df_dev["target_sentence"] = df_dev["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_dev.tail(num_dev_patterns) # Herman: Error in original
stripped = df_dev.drop(df_dev.tail(num_dev_patterns).index)

with open(f"{gdrive_path}/train."+source_language, "w") as src_file, open(f"{gdrive_path}/train."+target_language, "w") as tgt_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    tgt_file.write(row["target_sentence"]+"\n")
    
with open(f"{gdrive_path}/dev."+source_language, "w") as src_file, open(f"{gdrive_path}/dev."+target_language, "w") as tgt_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    tgt_file.write(row["target_sentence"]+"\n")

CPU times: user 2min 30s, sys: 446 ms, total: 2min 30s
Wall time: 2min 31s


In [0]:
# create my own test set

num_test_patterns = 1000

test = df_dev.tail(2000).iloc[0:num_test_patterns] # Herman
test.info()
test.tail()

# remove data leakage from test set- provided all dataframes used do not have NaNs or duplicates
filterd_test = test[~test.isin(df_dev)].dropna()

if len(filterd_test)<1:
    filterd_test=test
else:
    pass

filterd_test.info()
filterd_test.tail()

with open(f"{gdrive_path}/test2."+source_language, "w") as src_file, open(f"{gdrive_path}/test2."+target_language, "w") as tgt_file:
  for index, row in test.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 956286 to 957285
Data columns (total 2 columns):
source_sentence    1000 non-null object
target_sentence    1000 non-null object
dtypes: object(2)
memory usage: 15.8+ KB


,source_sentence,target_sentence
957281,"Shortly thereafter , my wife and I received a ...","Ngokushesha ngemva kwalokho , mina nomkami sat..."
957282,Soon I came to realize that learning what the ...,Ngokushesha ngabona ukuthi ukufunda lokho okuf...
957283,"If you have children , how can you help them ?","Uma unabantwana , ungabasiza kanjani ?"
957284,"With pleasure and interest , I read the magazi...",Ngenjabulo nesithakazelo ngawufunda lomagazini...
957285,After investigating all the details of Luke’s ...,Ngemva kokuhlola yonke imininingwane yohambo l...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 956286 to 957285
Data columns (total 2 columns):
source_sentence    1000 non-null object
target_sentence    1000 non-null object
dtypes: object(2)
memory usage: 15.8+ KB


,source_sentence,target_sentence
957281,"Shortly thereafter , my wife and I received a ...","Ngokushesha ngemva kwalokho , mina nomkami sat..."
957282,Soon I came to realize that learning what the ...,Ngokushesha ngabona ukuthi ukufunda lokho okuf...
957283,"If you have children , how can you help them ?","Uma unabantwana , ungabasiza kanjani ?"
957284,"With pleasure and interest , I read the magazi...",Ngenjabulo nesithakazelo ngawufunda lomagazini...
957285,After investigating all the details of Luke’s ...,Ngemva kokuhlola yonke imininingwane yohambo l...


115

...

39

92

97

175

215

In [0]:
# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head "$gdrive_path/train.$src"
! echo "================================="
! head "$gdrive_path/dev.$src"
! echo "================================="
! head "$gdrive_path/test.$src"

in any case , infants laid down either on their back or on their stomach tend to remain in that position 
□ how can elders help their brothers to serve jehovah with joy ?
here are some of the things such parents have in common 
upon defeating the amalekites , though , he disobeys jehovah by sparing what had been “ devoted to destruction  ”
the committees that were set up for korea are reporting good success 
we can see both aspects ​ — oath and friendship — ​ as the keystone of jehovah’s covenants 
evidently , people are interested in angels 
i turned to some boy for love  ”
called the song of solomon , it is the bible’s account of a beautiful shulammite girl and her love for a shepherd boy 
as joakim the wood - carver said : ‘ tomorrow the sun will come up again 
estienne , however , had the favor and protection of the king 
he attended a meeting of jehovah’s witnesses and was given a copy of the new world translation 
mapping paradise , a modern history book , contains details of mor

In [0]:
! head "$gdrive_path/train.$tgt"
! echo "================================="
! head "$gdrive_path/dev.$tgt"
! echo "================================="
! head "$gdrive_path/test.$tgt"

noma kunjalo , lapho izinsana zilaliswe ngomhlane noma ngesisu azivamile ukuphenduka 
□ abadala bangabasiza kanjani abafowabo ukuba bakhonze ujehova ngenjabulo ?
nazi ezinye zezinto abazali abanjalo abavumelana ngazo 
nokho , ngemva kokuba esenqobé ama - amaleki , akamlaleli ujehova ngokusindisa lokho okufanele ngabe ‘ kunikelwe ekubhujisweni  ’
amakomiti amiselwa ikorea abika impumelelo enhle 
singazibona zombili lezici — isifungo nobungane — njengensika yezivumelwano zikajehova 
ngokusobala , abantu banesithakazelo ezingelosini 
ngaphendukela komunye umfana ukuze ngithole uthando  ”
ibizwa ngokuthi isihlabelelo sezihlabelelo , iwukulandisa kwebhayibheli ngentombi enhle yomshulamiti nangothando lwayo ngomalusi wayo 
njengoba nje ujoakim umbazi athi : ‘ kusasa ilanga lizophuma futhi 
nokho , u - estienne wazuza umusa nesivikelo senkosi 
yaya emhlanganweni wofakazi bakajehova futhi yanikwa ikhophi ye - nguqulo yezwe elisha 
incwadi yomlando i - mapping paradise , inemininingwane yamabal

## Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [0]:
%%time
! subword-nmt learn-joint-bpe-and-vocab --input  "$gdrive_path"train.$src  "$gdrive_path"train.$tgt -s $vocab_size -o  "$gdrive_path"bpe.codes.$vocab_size --write-vocabulary  "$gdrive_path"vocab.$src  "$gdrive_path"vocab.$tgt

# Apply BPE splits to the train, development and test data.
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$src < "$gdrive_path"train.$src > "$gdrive_path"train.bpe.$src
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$tgt < "$gdrive_path"train.$tgt > "$gdrive_path"train.bpe.$tgt

! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$src < "$gdrive_path"dev.$src > "$gdrive_path"dev.bpe.$src
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$tgt < "$gdrive_path"dev.$tgt > "$gdrive_path"dev.bpe.$tgt

! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$src < "$gdrive_path"test2.$src > "$gdrive_path"test2.bpe.$src
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$tgt < "$gdrive_path"test2.$tgt > "$gdrive_path"test2.bpe.$tgt


CPU times: user 32.3 ms, sys: 45.2 ms, total: 77.5 ms
Wall time: 5.08 s


In [0]:
# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py "$gdrive_path"train.bpe."$src" "$gdrive_path"train.bpe."$tgt" --output_path "$gdrive_path"vocab.txt

In [0]:
# Some output
! echo "BPE Zulu Sentences"
! tail -n 5 "$gdrive_path"test.bpe.$tgt

! echo "Combined BPE Vocab"
! tail -n 10 "$gdrive_path"vocab.txt  # Herman

# !cp joeynmt/data/$src$tgt$vocab_size/vocab.txt "$gdrive_path"

BPE Zulu Sentences
N@@ g@@ oku@@ sh@@ esha ngemva kwalokho , mina nom@@ k@@ ami s@@ athola incwadi ev@@ ela eh@@ hov@@ isi leg@@ at@@ sha le - W@@ at@@ ch T@@ ower S@@ o@@ ci@@ ety isim@@ em@@ ela ukuba siye e@@ S@@ y@@ d@@ ne@@ y 
N@@ g@@ oku@@ sh@@ esha ngab@@ ona ukuthi ukufunda lokho oku@@ fund@@ iswa i@@ B@@ h@@ ayibheli ku@@ hamb@@ isana nem@@ ith@@ walo yem@@ fanelo ethile , ng@@ abe seng@@ inq@@ uma uku@@ yeka ukul@@ ifunda 
U@@ m@@ a un@@ abantwana , ung@@ abas@@ iza kanjani ?
N@@ gen@@ jabulo nes@@ ith@@ akazelo ng@@ aw@@ u@@ funda lom@@ agazini , futhi ng@@ ifuna ukwazi okwengeziwe ng@@ e@@ B@@ h@@ ayibheli 
N@@ g@@ em@@ va koku@@ hl@@ ola yonke imin@@ in@@ ing@@ wane yo@@ h@@ ambo luk@@ a@@ L@@ uka ngom@@ kh@@ umbi , it@@ il@@ osi u@@ J@@ ames S@@ m@@ ith l@@ aph@@ etha ngokuthi : “ L@@ okhu kuw@@ ukul@@ andisa kwez@@ enzakalo ezing@@ ok@@ oqobo , ez@@ ibh@@ al@@ we ng@@ othile oway@@ eh@@ il@@ eleke ngokom@@ untu siqu kuzo   
Combined BPE Vocab
א@@
ị@@
·
(@@
̆
ṇ@@
ז@@
{
ɓ@

## Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
name = '%s%s%s%s' % (source_language, target_language, str(vocab_size),tag)
name

'enzu4000baseline'

In [0]:
# Create this dir before we run for the first time so we store check points
# !mkdir -p "$gdrive_path/pretrained/$src$tgt$vocab_size$tag/" # Herman

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s%s%s' % (source_language, target_language, str(vocab_size),tag)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{gdrive_path}train.bpe"
    dev:   "{gdrive_path}dev.bpe"
    test:  "{gdrive_path}test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "{gdrive_path}vocab.txt"
    trg_vocab: "{gdrive_path}vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "{gdrive_path}pretrained/{name}/125000.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42                 # set this seed to make training deterministic
    optimizer: "adam"               # choices: "sgd", "adam", "adadelta", "adagrad", "rmsprop", default is SGD
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    # scheduling: "noam"           # TODO: try switching from plateau to Noam scheduling
    # patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    # learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    # learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 2048               # mini-batch size as number of sentences (when batch_type is "sentence"; default) or total number of tokens (when batch_type is "token")
    batch_type: "token"
    eval_batch_size: 1024
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 2500          # validate after this many updates (number of mini-batches), default: 1000
    logging_freq: 100              # log the training progress after this many updates, default: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "uniform"    # initializer for bias terms (xavier, zeros, normal, uniform)
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # size of embeddings (for Transformer set equal to hidden_size), TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.5
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.5
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.2
""".format(name=name,
           gdrive_path=os.environ["gdrive_path"],
           source_language=source_language,
           target_language=target_language
          )

with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

3863

## Train the Model

In [0]:
! cd joeynmt/configs; ls
# copy config to gdrive
! cp joeynmt/configs/transformer_$src$tgt$vocab_size$tag.yaml "$gdrive_path/pretrained/$src$tgt$vocab_size$tag/"

iwslt14_deen_bpe.yaml		   transformer_iwslt14_deen_bpe.yaml
iwslt_deen_bahdanau.yaml	   transformer_reverse.yaml
iwslt_envi_luong.yaml		   transformer_small.yaml
iwslt_envi_xnmt.yaml		   transformer_wmt17_ende.yaml
reverse.yaml			   transformer_wmt17_lven.yaml
small.yaml			   wmt_ende_best.yaml
transformer_copy.yaml		   wmt_ende_default.yaml
transformer_enzu4000baseline.yaml  wmt_lven_best.yaml
transformer_enzu4000const_lr.yaml  wmt_lven_default.yaml


In [0]:
%%time
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
! cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt$vocab_size$tag.yaml

2019-11-18 08:09:23,706 Hello! This is Joey-NMT.
2019-11-18 08:09:25,070 Total params: 12175104
2019-11-18 08:09:25,071 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
# !mkdir "$gdrive_path""pretrained/$src$trg$vocab_size$tag""2/"
!cp  -r joeynmt/models/${src}${trg}${vocab_size}${tag}_transformer/* "$gdrive_path""pretrained/$src$trg$vocab_size$tag/"
!cp  joeynmt/models/${src}${trg}${vocab_size}${tag}_transformer/best.ckpt "$gdrive_path""pretrained/$src$trg$vocab_size$tag/"

cp: cannot create symbolic link '/content/drive/My Drive/masakhane/en-zu-const_lr/pretrained/enzu4000const_lr2/best.ckpt': Function not implemented


In [0]:
# copy across the config file
!cp  joeynmt/configs/transformer_${src}${trg}${vocab_size}${tag}.yaml "$gdrive_path"

## Test our model

In [0]:
# ! cd joeynmt; python3 -m joeynmt test "$gdrive_path""pretrained/$src$trg$vocab_size$tag/config.yaml"

2019-11-13 13:50:15,142 Hello! This is Joey-NMT.
2019-11-13 13:50:45,623  dev bleu:  11.87 [Beam search decoding with beam size = 5 and alpha = 1.0]
2019-11-13 13:50:57,050 test bleu:   5.00 [Beam search decoding with beam size = 5 and alpha = 1.0]


In [0]:
# ! cd joeynmt; python3 -m joeynmt test "$gdrive_path""transformer_${src}${trg}${vocab_size}${tag}.yaml"

2019-11-13 07:43:32,728 Hello! This is Joey-NMT.
2019-11-13 07:44:03,502  dev bleu:  13.64 [Beam search decoding with beam size = 5 and alpha = 1.0]
2019-11-13 07:44:24,289 test bleu:   4.87 [Beam search decoding with beam size = 5 and alpha = 1.0]


In [0]:
# Translate mode is mopre interactive but almsot the same as running in test mode
! cd joeynmt; python3 -m joeynmt translate "$gdrive_path""transformer_${src}${trg}${vocab_size}${tag}.yaml"

2019-11-11 14:07:11,216 Hello! This is Joey-NMT.

Please enter a source sentence (pre-processed): 
Hello and welcome, my name is Ari
JoeyNMT: futhi igama lami liwusizo

Please enter a source sentence (pre-processed): 
Thank you for contacting customer service
JoeyNMT: uqondisa inkonzo

Please enter a source sentence (pre-processed): 
How can I be of assistance ?
JoeyNMT: ingabe kungenzeka yini ukuba khona ?

Please enter a source sentence (pre-processed): 

Bye.


In [0]:
# Output our validation accuracy
! cat "$gdrive_path/pretrained/${src}${trg}${vocab_size}${tag}/validations.txt"

Steps: 98000	Loss: 62210.42969	PPL: 10.84522	bleu: 10.83698	LR: 0.00030000	*
Steps: 99000	Loss: 62231.77734	PPL: 10.85409	bleu: 10.96018	LR: 0.00030000	
Steps: 100000	Loss: 62221.71875	PPL: 10.84991	bleu: 10.90364	LR: 0.00030000	
Steps: 101000	Loss: 62062.70312	PPL: 10.78400	bleu: 10.71851	LR: 0.00030000	*
Steps: 102000	Loss: 62021.57031	PPL: 10.76702	bleu: 10.91517	LR: 0.00030000	*
Steps: 103000	Loss: 61928.83984	PPL: 10.72883	bleu: 10.72367	LR: 0.00030000	*
Steps: 104000	Loss: 61953.24609	PPL: 10.73887	bleu: 10.99511	LR: 0.00030000	
Steps: 105000	Loss: 62004.41406	PPL: 10.75994	bleu: 11.06624	LR: 0.00030000	
Steps: 106000	Loss: 61889.42188	PPL: 10.71264	bleu: 10.91160	LR: 0.00030000	*
Steps: 107000	Loss: 61763.37500	PPL: 10.66102	bleu: 10.85862	LR: 0.00030000	*
Steps: 108000	Loss: 61857.72266	PPL: 10.69963	bleu: 10.98885	LR: 0.00030000	
Steps: 109000	Loss: 61657.77344	PPL: 10.61797	bleu: 11.11039	LR: 0.00030000	*
Steps: 110000	Loss: 61585.40625	PPL: 10.58857	bleu: 10.77238	LR: 0.0003

In [0]:
! ls "$gdrive_path"

bpe.codes.4000	     test.bpe.en     train.en
dev.bpe.en	     test.bpe.zu     train.zu
dev.bpe.zu	     test.en	     transformer_enzu4000const_lr.yaml
dev.en		     test.en-any.en  vocab.en
dev.zu		     test.zu	     vocab.txt
en-zu-JW300-new.csv  train.bpe.en    vocab.zu
pretrained	     train.bpe.zu


## Check vocab coverage in test set


### Vocab

In [0]:
# get BPE vocab
filename =  f"{gdrive_path}/vocab.txt"
with open(filename) as f:
    vocab_tokens = [tok.strip() for tok in f]

vocab_tokens_set = set(vocab_tokens)

In [0]:
f'There are {len(vocab_tokens)} tokens in the combined vocab, {len(vocab_tokens_set)} are unique'

'There are 4351 tokens in the combined vocab, 4351 are unique'

### Training

In [0]:
# get training BPE vocab
en_filename =  f"{gdrive_path}/train.bpe.en"
with open(en_filename) as f:
    en_train_tokens = [tok for line in f for tok in line.split()]

zu_filename =  f"{gdrive_path}/train.bpe.zu"
with open(zu_filename) as f:
    zu_train_tokens = [tok for line in f for tok in line.split()]

en_train_tokens_set = set(en_train_tokens)
zu_train_tokens_set = set(zu_train_tokens)


In [0]:
print(f'There are {len(en_train_tokens)} english tokens in the training set vocab, {len(en_train_tokens_set)} are unique ')
print(f'There are {len(zu_train_tokens)} zulu tokens in the training set vocab, {len(zu_train_tokens_set)} are unique')
print(f'Thats a uniqueness ratio of {len(en_train_tokens_set)/len(en_train_tokens) :.2E} for english and {len(zu_train_tokens_set)/len(zu_train_tokens) :.2E} for zulu')

There are 24094443 english tokens in the training set vocab, 3112 are unique 
There are 24223704 zulu tokens in the training set vocab, 4188 are unique
Thats a uniqueness ratio of 1.29E-04 for english and 1.73E-04 for zulu


### Dev

In [0]:
# get dev BPE vocab
en_filename =  f"{gdrive_path}/dev.bpe.en"
with open(en_filename) as f:
    en_dev_tokens = [tok for line in f for tok in line.split()]

zu_filename =  f"{gdrive_path}/dev.bpe.zu"
with open(zu_filename) as f:
    zu_dev_tokens = [tok for line in f for tok in line.split()]

en_dev_tokens_set = set(en_dev_tokens)
zu_dev_tokens_set = set(zu_dev_tokens)

In [0]:
print(f'There are {len(en_dev_tokens)} english tokens in the dev set vocab, {len(en_dev_tokens_set)} are unique ')
print(f'There are {len(zu_dev_tokens)} zulu tokens in the dev set vocab, {len(zu_dev_tokens_set)} are unique')
print(f'Thats a uniqueness ratio of {len(en_dev_tokens_set)/len(en_dev_tokens) :.2f} for english and {len(zu_dev_tokens_set)/len(zu_dev_tokens) :.2f} for zulu')

There are 24651 english tokens in the dev set vocab, 2104 are unique 
There are 25098 zulu tokens in the dev set vocab, 2418 are unique
Thats a uniqueness ratio of 0.09 for english and 0.10 for zulu


### Test

In [0]:
# get test BPE vocab
en_filename =  f"{gdrive_path}/test.bpe.en"
with open(en_filename) as f:
    en_test_tokens = [tok for line in f for tok in line.split()]


zu_filename =  f"{gdrive_path}/test.bpe.zu"
with open(zu_filename) as f:
    zu_test_tokens = [tok for line in f for tok in line.split()]


en_test_tokens_set = set(en_test_tokens)
zu_test_tokens_set = set(zu_test_tokens)

In [0]:
print(f'There are {len(en_test_tokens)} english tokens in the test set vocab, {len(en_test_tokens_set)} are unique ')
print(f'There are {len(zu_test_tokens)} zulu tokens in the test set vocab, {len(zu_test_tokens_set)} are unique')
print(f'Thats a uniqueness ratio of {len(en_test_tokens_set)/len(en_test_tokens) :.2E} for english and {len(zu_test_tokens_set)/len(zu_test_tokens) :.2E} for zulu')

There are 66255 english tokens in the test set vocab, 2072 are unique 
There are 67851 zulu tokens in the test set vocab, 2336 are unique
Thats a uniqueness ratio of 3.13E-02 for english and 3.44E-02 for zulu


### Vocab Differences

In [0]:
test_en_not_in_vocab = vocab_tokens_set.difference(en_test_tokens_set)
test_en_IS_in_vocab = vocab_tokens_set.intersection(en_test_tokens_set)

test_zu_not_in_vocab = vocab_tokens_set.difference(zu_test_tokens_set)
test_zu_IS_in_vocab = vocab_tokens_set.intersection(zu_test_tokens_set)

In [0]:
print(f'There are {len(test_en_IS_in_vocab)} tokens from vocab in the en test set  -> {100 *len(test_en_IS_in_vocab)/len(vocab_tokens_set) :.3f}% Coverage')
print(f'There are {len(test_zu_IS_in_vocab)} tokens from vocab in the en test set  -> {100 *len(test_zu_IS_in_vocab)/len(vocab_tokens_set) :.3f}% Coverage')

There are 2003 tokens from vocab in the en test set  -> 46.035% Coverage
There are 2269 tokens from vocab in the en test set  -> 52.149% Coverage


In [0]:
print(f'There are {len(test_en_not_in_vocab)} tokens from vocab in the en test set  -> {100 *len(test_en_not_in_vocab)/len(vocab_tokens_set) :.3f}% Missing')
print(f'There are {len(test_zu_not_in_vocab)} tokens from vocab in the en test set  -> {100 *len(test_zu_not_in_vocab)/len(vocab_tokens_set) :.3f}% Missing')

There are 2348 tokens from vocab in the en test set  -> 53.965% Missing
There are 2082 tokens from vocab in the en test set  -> 47.851% Missing


In [0]:
# Overall coverage: Remember that our vocab is combined eng/zulu so we're expecting ~50% each
global_coverage =list(test_en_IS_in_vocab.union(test_zu_IS_in_vocab))
global_diff = list(test_en_not_in_vocab.difference(test_zu_IS_in_vocab))

In [0]:
100*len(global_coverage)/len(vocab_tokens_set)

84.78510687198346

In [0]:
100*len(global_diff)/len(vocab_tokens_set)

15.214893128016548

### What are some of the missing tokens ?



In [0]:
global_diff[0:10]

['ᾶ@@', 'ẹ', 'ã@@', 'isambulo', 'elapha', 'juda', 'ì', 'temple', 'ū@@', 'ḥ@@']